<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/NLP/4-5-Llamaindex02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community openai chromadb tiktoken

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader

In [3]:
from google.colab import userdata
import os

userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
!mkdir data

In [5]:
!wget https://github.com/JSJeong-me/GPT-Web/raw/main/images/paul_graham_essay.txt -O ./data/paul_graham_essay.txt

--2024-10-07 07:52:49--  https://github.com/JSJeong-me/GPT-Web/raw/main/images/paul_graham_essay.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JSJeong-me/GPT-Web/main/images/paul_graham_essay.txt [following]
--2024-10-07 07:52:49--  https://raw.githubusercontent.com/JSJeong-me/GPT-Web/main/images/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘./data/paul_graham_essay.txt’

./data/paul_graham_ 100%[===================>]  73.28K  --.-KB/s    in 0.003s  

2024-10-07 07:52:49 (27.6 MB/s) - ‘./data/paul_graham_essay.txt’ saved [750

In [6]:
# Step 1: 문서 로드
# 여기에 분석하고자 하는 텍스트 파일을 넣으세요.
loader = TextLoader('./data/paul_graham_essay.txt')
documents = loader.load()


In [7]:
# Step 2: 텍스트 분할
from langchain.text_splitter import CharacterTextSplitter # import the CharacterTextSplitter class

# 문서를 최대 1000 토큰의 청크로 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # reduced chunk size to 1000
documents = text_splitter.split_documents(documents)

# Step 3: 임베딩 및 Chroma 벡터 스토어 생성
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

# Step 4: 검색 및 QA 체인 생성
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}) # only retrieve the top 2 most relevant documents
)

# Step 5: 질문에 답변
query = "What is the author name?"
answer = qa.run(query)
print(f"Answer: {answer}")

<ipython-input-7-a6dba6db56d0>:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
<ipython-input-7-a6dba6db56d0>:14: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(),
<ipython-input-7-a6dba6db56d0>:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answ

Answer:  Paul Graham


In [9]:
query = "How many legs does a cat have?"
answer = qa.run(query)
print(f"Answer: {answer}")

Answer:  I don't know.
